In [24]:
import pandas as pd

# Load the datasets
balance_df = pd.read_csv('/Users/ansonkong/Downloads/Data for nyu work/Input/EIA 930/BA/EIA930_BALANCE_2020_Jan_Jun.csv')
subregion_df = pd.read_csv('/Users/ansonkong/Downloads/Data for nyu work/Input/EIA 930/Subregion/EIA930_SUBREGION_2020_Jan_Jun.csv')

# Filter the datasets
balance_df_filtered = balance_df[(balance_df['UTC Time at End of Hour'] == '02/01/2020 7:00:00 AM')& (balance_df['Demand (MW)'].notna())]
subregion_df_filtered = subregion_df[(subregion_df['UTC Time at End of Hour'] == '02/01/2020 7:00:00 AM') & (subregion_df['Demand (MW)'].notna())]

# Get the number of distinct Sub-Region and Balancing Authority for SUBREGION
distinct_sub_region = subregion_df_filtered['Sub-Region'].nunique()
distinct_ba_subregion = subregion_df_filtered['Balancing Authority'].nunique()

# For BALANCE, find distinct Balancing Authority not in SUBREGION
distinct_ba_balance = balance_df_filtered[~balance_df_filtered['Balancing Authority'].isin(subregion_df_filtered['Balancing Authority'])]['Balancing Authority'].nunique()

print(f"Number of distinct Sub-Region: {distinct_sub_region}")
print(f"Number of distinct Balancing Authority in SUBREGION: {distinct_ba_subregion}")
print(f"Number of distinct Balancing Authority in BALANCE not in SUBREGION: {distinct_ba_balance}")


/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_15603/598961565.py:4: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  balance_df = pd.read_csv('/Users/ansonkong/Downloads/Data for nyu work/Input/EIA 930/BA/EIA930_BALANCE_2020_Jan_Jun.csv')


Number of distinct Sub-Region: 82
Number of distinct Balancing Authority in SUBREGION: 8
Number of distinct Balancing Authority in BALANCE not in SUBREGION: 45


In [2]:
#pre define path
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'

In [3]:
import pandas as pd
import os

# Paths to the directories
subregion_dir = data_and_output_dir+'Input/EIA 930/Subregion/'
ba_dir = data_and_output_dir+'Input/EIA 930/BA/'

# Function to read and merge CSVs from a directory
def read_and_merge_csvs(directory, cols):
    combined_df = pd.DataFrame()
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file), usecols=cols)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# Columns to keep for each dataset
subregion_cols = ['UTC Time at End of Hour', 'Demand (MW)', 'Sub-Region', 'Balancing Authority']
ba_cols = ['UTC Time at End of Hour', 'Balancing Authority', 'Demand (MW)']

# Read and merge CSVs
subregion_combined_df = read_and_merge_csvs(subregion_dir, subregion_cols)
ba_combined_df = read_and_merge_csvs(ba_dir, ba_cols)

# Before the loop, initialize an empty list to collect new rows
new_rows = []

# Iterate over BA combined dataframe to check for missing Balancing Authorities in Subregion dataframe
for index, row in ba_combined_df.iterrows():
    matching_rows = subregion_combined_df[(subregion_combined_df['UTC Time at End of Hour'] == row['UTC Time at End of Hour']) & 
                                           (subregion_combined_df['Balancing Authority'] == row['Balancing Authority'])]
    if matching_rows.empty:
        # If no matching row, prepare the new row with adjustments
        new_row = row.to_dict()
        new_row['Sub-Region'] = new_row['Balancing Authority']  # Set Sub-Region to Balancing Authority value
        new_rows.append(new_row)

# After the loop, add all new rows to the dataframe at once
if new_rows:  # Check if there are any new rows to add
    subregion_combined_df = pd.concat([subregion_combined_df, pd.DataFrame(new_rows)], ignore_index=True)

# You may now use subregion_combined_df with the added records.
# Optionally, you can save this dataframe to a new CSV file
subregion_combined_df.to_csv(data_and_output_dir+'output/merged_subregion_with_ba.csv', index=False)



/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_1417/3776213816.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_1417/3776213816.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_1417/3776213816.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_1417/3776213816.py:13: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/

In [32]:
import geopandas as gpd
import pandas as pd

# Assuming data_and_output_dir is defined and contains the path to your data directory
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/' # Update this path

# Path to the shapefile and CSV
shapefile_path = data_and_output_dir + 'Input/cb_2018_us_county_500k/cb_2018_us_county_500k.shp'
csv_path = data_and_output_dir + 'Input/ba_service_territory_2020.csv'

# Read the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

# Read the CSV file. Use GeoPandas if you expect to work with geographic data in the CSV.
# If the CSV doesn't contain geographic data, you could just use pandas with pd.read_csv(csv_path)
df = pd.read_csv(csv_path)

# Print all columns from the shapefile
print("Columns in the shapefile:")
print(gdf.columns)

# Print all columns from the CSV file
print("\nColumns in the CSV file:")
print(df.columns)


Columns in the shapefile:
Index(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry'],
      dtype='object')

Columns in the CSV file:
Index(['Year', 'State_FIPS', 'State_Name', 'County_FIPS', 'County_Name',
       'BA_Number', 'BA_Code'],
      dtype='object')


In [4]:
import numpy as np
data = np.load('/Users/ansonkong/Downloads/2007.npy')

In [6]:
import numpy as np

# Load the data
data = np.load('/Users/ansonkong/Downloads/2007.npy')

# Check the shape of the loaded array
print("Data shape:", data.shape)

# Assuming the first axis represents time and the second represents p_zones
num_hours, num_p_zones = data.shape

# Validate the time dimension
print("Number of hours:", num_hours)
if num_hours == 8760:
    print("Time dimension is correct for a non-leap year.")
else:
    print("Time dimension mismatch. Expected 8760 for a non-leap year.")

# Check the ordering and count of p_zones
print("Number of p_zones:", num_p_zones)


Data shape: (8760, 134)
Number of hours: 8760
Time dimension is correct for a non-leap year.
Number of p_zones: 134


In [10]:
import pandas as pd
# Create p_zone labels
p_zone_labels = [f'p{i}' for i in range(1, 135)]
# Correctly ordering p_zone_labels numerically
p_zone_labels_sorted = sorted(p_zone_labels, key=lambda x: x)

# Creating a time index for the DataFrame
time_index = pd.date_range(start='2007-01-01', periods=data.shape[0], freq='H')

# Constructing the DataFrame with correctly ordered p_zone_labels
df = pd.DataFrame(data, index=time_index, columns=p_zone_labels_sorted)

# Displaying the first few rows of the DataFrame to check its structure
print(df.head())

                         p1      p10   p100     p101    p102    p103   p104  \
2007-01-01 00:00:00  5317.0  12190.0  256.0  11816.0  6722.0  8051.0  766.0   
2007-01-01 01:00:00  5122.0  11580.0  243.0  11178.0  6359.0  7907.0  746.0   
2007-01-01 02:00:00  5025.0  11101.0  237.0  10339.0  5881.0  7768.0  733.0   
2007-01-01 03:00:00  4992.0  10804.0  237.0   9877.0  5618.0  7524.0  724.0   
2007-01-01 04:00:00  5088.0  10690.0  241.0   9614.0  5469.0  7442.0  731.0   

                       p105    p106    p107  ...     p90    p91     p92  \
2007-01-01 00:00:00  5410.0  1464.0  2317.0  ...  3340.0  952.0  8834.0   
2007-01-01 01:00:00  5264.0  1427.0  2254.0  ...  3256.0  926.0  8773.0   
2007-01-01 02:00:00  5131.0  1390.0  2207.0  ...  3201.0  909.0  8708.0   
2007-01-01 03:00:00  5098.0  1376.0  2172.0  ...  3185.0  904.0  8714.0   
2007-01-01 04:00:00  5123.0  1376.0  2165.0  ...  3218.0  912.0  8865.0   

                        p93      p94     p95     p96     p97     p98     p

/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_15853/3972925922.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start='2007-01-01', periods=data.shape[0], freq='H')
